### Checklist

* Re-name header to ROI
* The script and the csv file has to be in the same folder, if not modify path
* Enter correct values for baseline and treatments
* For all cells with "plot_" manually select the ROIs 
* Cross-check logic in cells where data frames are being merged
* Change labels appropriately in summary table
* Name the export sheets appropriately


In [1]:
import pandas as pd
import openpyxl 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
#import csv
raw = "/Users/dalegeorge/Desktop/In vivo calcium/RD_concatenated_8_9_10_12_13.xlsx"
raw_df = pd.read_excel(raw)
raw_df.head(2)

,Time (sec),ROI1,ROI2,ROI3,ROI4,ROI5,ROI6,ROI7,ROI8,ROI9,ROI10,ROI11,ROI12,ROI13,ROI14,ROI15,ROI16
0,1,0.069,-0.057,0.042,-0.089,0.006,0.124,0.052,-0.056,0.028,0.04,-0.020,0.051,0.051,0.017,-0.007,-0.016
1,2,-0.148,-0.009,0.008,0.175,-0.028,0.043,0.152,-0.016,0.047,0.03,0.037,-0.031,0.085,0.101,0.028,-0.065


In [3]:
# Defining the functions

def Baseline(start, end):
    raw_df = pd.read_excel(raw)
    baseline = raw_df.loc[(raw_df["Time (sec)"] >= start) & (raw_df["Time (sec)"] <= end)]
    baseline = baseline.set_index('Time (sec)')
    baseline = np.std(baseline)
    return baseline


def Responders(start, end):   
#Selecting the duration of treatment
    duration = raw_df.loc[(raw_df["Time (sec)"] >= start) & (raw_df["Time (sec)"] <= end)]
    duration = duration.set_index("Time (sec)")
    duration_max = duration.max()
    
#Determining responders
    responders = duration_max > 6*baseline
    responders = responders.transpose().reset_index()
    responders.columns=["cells", "response"]
    responders = responders.loc[responders["response"] == True]
    return responders

In [4]:
#Treatment 1
baseline = Baseline(0, 19) # first 20 seconds of the treatment as baseline
t1_responders = Responders(20, 100)
t1_responders

,cells,response
4,ROI5,True
9,ROI10,True
15,ROI16,True


In [5]:
plot_t1 = raw_df[["Time (sec)", \
                        "ROI5", "ROI10", "ROI16"]]

In [6]:
#Treatment 2
baseline = Baseline(101, 119)

t2_responders = Responders(120, 200)
t2_responders

,cells,response
0,ROI1,True
2,ROI3,True
6,ROI7,True
8,ROI9,True
9,ROI10,True
12,ROI13,True
13,ROI14,True


In [7]:
plot_t2 = raw_df[["Time (sec)", \
                        "ROI1", "ROI3", "ROI7", "ROI9", "ROI10", "ROI13", "ROI14"]]

In [8]:
#Treatment 3
baseline = Baseline(201, 219)

t3_responders = Responders(220, 300)
t3_responders

,cells,response
1,ROI2,True
2,ROI3,True
9,ROI10,True


In [9]:
plot_t3 = raw_df[["Time (sec)", \
                        "ROI2", "ROI3", "ROI10"]]

In [10]:
#Treatment 4
baseline = Baseline(301, 319)

t4_responders = Responders(320, 400)
t4_responders

,cells,response
2,ROI3,True
4,ROI5,True
10,ROI11,True
14,ROI15,True


In [11]:
plot_t4 = raw_df[["Time (sec)", \
                        "ROI3", "ROI5", "ROI11", "ROI15"]]

In [12]:
#cells that respond to beta-alanine but not saline
only_ba = pd.merge(t2_responders, t1_responders, on='cells', how='outer')
only_ba = only_ba.rename(columns={"response_x": "Beta-alanine", "response_y": "Saline"}).dropna(subset=["Beta-alanine"])
only_ba

,cells,Beta-alanine,Saline
0,ROI1,True,NaN
1,ROI3,True,NaN
2,ROI7,True,NaN
3,ROI9,True,NaN
4,ROI10,True,True
5,ROI13,True,NaN
6,ROI14,True,NaN


In [13]:
#Removing cells that responded to both treatments
only_ba = only_ba.drop([4]) #index the row where both columns have a true value
only_ba

,cells,Beta-alanine,Saline
0,ROI1,True,NaN
1,ROI3,True,NaN
2,ROI7,True,NaN
3,ROI9,True,NaN
5,ROI13,True,NaN
6,ROI14,True,NaN


In [45]:
plot_only_ba = raw_df[["Time (sec)", \
                        "ROI1", "ROI3", "ROI7", "ROI9", "ROI13", "ROI14"]]

In [14]:
#cells that repsond to capsaicin but not ethanol
only_cap = pd.merge(t4_responders, t3_responders, on='cells', how='outer')
only_cap = only_cap.rename(columns={"response_x": "Capsaicin", "response_y": "Ethanol"}).dropna(subset=["Capsaicin"])
only_cap

,cells,Capsaicin,Ethanol
0,ROI3,True,True
1,ROI5,True,NaN
2,ROI11,True,NaN
3,ROI15,True,NaN


In [15]:
#Removing cells that responded to both treatments
only_cap = only_cap.drop([0]) #index the row where both columns have a true value
only_cap

,cells,Capsaicin,Ethanol
1,ROI5,True,NaN
2,ROI11,True,NaN
3,ROI15,True,NaN


In [46]:
plot_only_cap = raw_df[["Time (sec)", \
                        "ROI5", "ROI11", "ROI15"]]

In [20]:
#cells that responded to both capsaicin and any beta-alanine
ba_cap = pd.merge(only_ba[["cells","Beta-alanine"]], only_cap[["cells","Capsaicin"]], on="cells", how="inner")
ba_cap

,cells,Beta-alanine,Capsaicin


In [48]:
plot_ba_cap = raw_df["Time (sec)"]

In [42]:
#Determining % of reponders

area_Nav_GCaMP = 0.00051

area_of_DRG = float(input("What is the area?: "))


no_of_neurons = area_of_DRG * area_Nav_GCaMP


per_t1_responders = round((t1_responders["cells"].count()/no_of_neurons)*100, 2)
per_t3_responders = round((t3_responders["cells"].count()/no_of_neurons)*100, 2)

per_only_ba_responders = round((only_ba["cells"].count()/no_of_neurons)*100, 2)
per_only_cap_responders = round((only_cap["cells"].count()/no_of_neurons)*100, 2)
per_ba_cap_responders = round((ba_cap["cells"].count()/no_of_neurons)*100, 2)


What is the area?: 0.0002


In [43]:
treatment_summary = pd.DataFrame({"Saline": [per_t1_responders],
                      "Beta alanine (100mM)": [per_only_ba_responders],
                      "Ethanol": [per_t3_responders],
                      "Capsaicin (10 mM)": [per_only_cap_responders],
                      "Beta alanine and capsaicin": [per_ba_cap]})

treatment_summary


,Saline,Beta alanine (100mM),Ethanol,Capsaicin (10 mM),Beta alanine and capsaicin
0,2.941176e+09,5.882353e+09,2.941176e+09,2.941176e+09,0.0


In [49]:
# Exporting files

with pd.ExcelWriter('output.xlsx') as writer:  
    plot_t1.to_excel(writer, sheet_name='t1', index=False)
    plot_t2.to_excel(writer, sheet_name='t2', index=False)
    plot_t3.to_excel(writer, sheet_name='t3', index=False)
    plot_t4.to_excel(writer, sheet_name='t4', index=False)
    plot_only_ba.to_excel(writer, sheet_name='only_ba', index=False)
    plot_only_cap.to_excel(writer, sheet_name='only_cap', index=False)
    plot_ba_cap.to_excel(writer, sheet_name='ba_cap', index=False)    